In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:

import configparser, os

import huggingface_hub, wandb

config = configparser.ConfigParser()

config.read('../../config/config.ini')

SECRETS = config['SECRETS']

# set openai api key

os.environ['OPENAI_API_KEY'] = SECRETS['openapi_key']

wandb.login(key = SECRETS['wandb_key'])

huggingface_hub.login(token = SECRETS['hf_key'])


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: n-suneel89 (n-suneel-duke). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\91939\.netrc
c:\Me\Duke\Classes\Spring '24\AIPI 540 - Deep Learning\Individual Project\workspace\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\91939\.cache\huggingface\token
Login successful


In [8]:
import os
import json
import re

import openai

import langchain

import langchain.document_loaders

from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.chroma import Chroma
import os
import shutil

from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

import sys 

sys.path.append('../../')

from scripts.summarization import rag_summ

In [4]:
def get_chunks(file_path):
    
    loader = PyPDFLoader(file_path)
    
    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100,
    length_function=len,
    add_start_index=True,
    )

    chunks = text_splitter.split_documents(documents)

    return chunks

In [5]:
def get_vectordb(chunks, CHROMA_PATH):

    CHROMA_PATH = f"../../data/chroma/{CHROMA_PATH}"

    if os.path.exists(CHROMA_PATH):
        db = Chroma(persist_directory=CHROMA_PATH, embedding_function=OpenAIEmbeddings())

    else:
        db = Chroma.from_documents(
            chunks, OpenAIEmbeddings(), persist_directory=CHROMA_PATH
        )

        db.persist()

        print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")
    
    return db

In [14]:
def gen_sample(text, db):

    PROMPT_TEMPLATE = """
    Answer the question based only on the following context:

    {context}

    ---

    Answer the question based on the above context: {question}
    """

    query_text = f"""

    Act as the author of a Choose Your Own Adventure Book. This book is special as it is based on existing material.
    Now, as with any choose your own adventure book, you'll have to generate decision paths at certain points in the story.
    Your job is to generate 4 decision paths for the given point in the story, if applicable to that point in the story.
    If the given part of the story doesn't contain any decisions from which to generate decision paths, don't
    generate any. If the given part of the story contains a decision, generate 4 decision paths for that decision.
    One among the 4 decision paths should be the original path, the other 3 should deviate from the original path in a sensible manner.
    The decision paths should be generated in a way that they are coherent with the existing story.
    The result should be a JSON object with the following keys: [text, paths]

    text: The given text
    paths: The generated decision paths as strings in a list

    ```{text}```

    """

    results = db.similarity_search_with_relevance_scores(query_text, k=5)

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)

    model = ChatOpenAI()
    response_text = model.predict(prompt)

    return eval(response_text)


In [9]:
from tqdm import tqdm

import random

base_path = "../../data/pdf"

samples = []

pdfs = os.listdir(base_path)

In [11]:
pdf = pdfs[0]

pages = rag_summ.read_pages(f"{base_path}/{pdf}")

CHROMA_PATH = f"{base_path}/{pdf}".split("/")[-1].split(".")[0]

chunks = get_chunks(f"{base_path}/{pdf}")

# print(f"Found {len(chunks)} chunks in {pdf_path}.")

db = get_vectordb(chunks, CHROMA_PATH)

c:\Me\Duke\Classes\Spring '24\AIPI 540 - Deep Learning\Individual Project\workspace\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [15]:
gen_sample(pages[20], db)

{'text': "20. Rabban stops outside the doorway. RABBAN Milord Baron. BARON HARKONNEN (O.S.) Rabban. The deep, resonant voice emerges from the steam. The vapor billows as something moves inside -- and we catch a glimpse of the steam bath’s occupant: the BARON VLADIMIR HARKONNEN. He is mountainous. Six hundred pounds of soft, naked flesh. Despite his vast bulk he considers himself beautiful. His movements are sensual. RABBAN The last of our ships have left Arrakis. It’s done. BARON HARKONNEN Very good. He waves Rabban away in a billow of steam. RABBAN Uncle. How can we let this happen? A fresh BLAST OF STEAM boils into the steam bath, drowning out Rabban’s question with a loud HISS. Rabban tries again. RABBAN(CONT'D) How can the Emperor give everything we’ve built to that Duke? PITER Don’t be too sure it’s an act of love. Rabban looks at Piter as though her were a strange insect. He replies to the Baron. RABBAN What does he mean? The Baron’s voice issues from the mist: BARON HARKONNEN Wh

In [16]:

for pdf in pdfs:

    print(f"Processing {pdf}... {pdfs.index(pdf)}/{len(pdfs)}")
    
    pdf_path = f"{base_path}/{pdf}"

    CHROMA_PATH = pdf_path.split("/")[-1].split(".")[0]

    chunks = get_chunks(pdf_path)

    pages = rag_summ.read_pages(pdf_path)

    db = get_vectordb(chunks, CHROMA_PATH)

    try:

        pages = random.sample(pages, 100)
    
    except:
        pages = pages

    for pages in tqdm(pages):

        try:

            sample = gen_sample(pages, db)

            samples.append(sample)
        except:
            continue
    
    print(f"\n{'-----'*50}\n")

Processing dune.pdf... 0/10


100%|██████████| 100/100 [15:28<00:00,  9.28s/it]



----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Processing eg.pdf... 1/10


100%|██████████| 100/100 [17:35<00:00, 10.56s/it]



----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Processing gr.pdf... 2/10


...


----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Processing nr.pdf... 8/10


100%|██████████| 100/100 [13:50<00:00,  8.31s/it]



----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Processing pokiri.pdf... 9/10


100%|██████████| 58/58 [09:25<00:00,  9.75s/it]


----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



In [17]:
samples

[{'text': 'Salmon Rev. (06/19/2020) 99. BARON HARKONNEN The Fremen? Kill them all. INT. ORNITHOPTER (IN FLIGHT) - DAY 125 125 The altimeter reads 8000 meters. On Paul. KYNES (V.O.) * Above five thousand meters, they’re * mostly dust. Climb into it. Stay on * top. You might just live. * PAUL (V.O.) * 8000 meters. We’re still climbing. * Dust has penetrated the cockpit. It swirls across the deck * and hangs in the air, laden with spice -- orange motes *drifting before his eyes. Jessica coughs in the dust. * PAUL (V.O.) * I’ll need to time the top of the * vortex. I can see it. It will be * there... In few seconds... I can * see it.... * Paul opens the wings. It takes all his strength. They creak and whine -- but they unfurl. EXT. DUST STORM - DAY 126 126 The little ornithopter breaks out of the vast storm, miles up. Banking away from the hurricane into blue skies. The aircraft’s glass bubble has been scoured to a milky translucence by dust. The wings are bent and trembling. The engines c

In [18]:
import pandas as pd

paths_df = pd.DataFrame(samples)

In [20]:
paths_df.head()

,text,paths
0,Salmon Rev. (06/19/2020) 99. BARON HARKONNEN T...,[Continue following the original path as Paul ...
1,PAUL *\nI could have died. *\nJESSICA *\nI kno...,[Paul decides to embrace his potential as ‘the...
2,GURNEY (O.S.) * (CONT'D) They say: it’s everyw...,[]
3,All part of a plan. (long beat) Do you... love...,[]
4,"INT. RESIDENCY, MASTER BEDROOM - NIGHT 78A 78A...",[Continue to observe the Suk School symbol and...


In [24]:
len(paths_df)

824

In [25]:
paths_df.to_csv("../../data/output/paths.csv", index=False)